Customers who use Snowflake with AzureML and Sagemaker will have models registered in either cloud ML tool and prefer to deploy those models to Snowflake. Often it's the case that all (or most) of the inference data is in Snowflake and customers prefer the ease of use, performance and security of having the model deployed in Snowflake.

For models registered in the AzureML model registry you can use the below code to access the model from the AzureML registry and push it to the Snowflake registry.

Important Notes

For both AzureML and Sagemaker deployment patterns users will need to make sure that they have properly installed all of the necessary libraries in their Python environments. For the required Snowflake packages this can usually be done with the instructions here: https://docs.snowflake.com/en/developer-guide/snowpark-ml/index#installing-snowpark-ml

Additionally, when using log_model() to push the model into the Snowflake registry users may need to specific dependencies and versions. This can be done with an additional argument in the function. Please see the documentation here: https://docs.snowflake.com/en/developer-guide/snowpark-ml/reference/latest/api/registry/snowflake.ml.registry.Registry#snowflake.ml.registry.Registry

Lastly, users will want to make sure that their model is supported by libraries that are included in the Snowflake conda channel. https://repo.anaconda.com/pkgs/snowflake/
If the model is not supported then users will need to consider installing the libraries as 3rd party packages or using Snowpark Container Services to deploy their model as an application. https://docs.snowflake.com/en/developer-guide/snowpark-container-services/overvie


In [ ]:
from azureml.core import Workspace, Dataset
import numpy as np

# Connect to the Workspace
ws = Workspace.from_config()

In [ ]:
# Access model using model name
from azureml.core import Model
model_path = Model.get_model_path(model_name = '<model name>', version = 1, _workspace= ws)
model_path

In [ ]:
# Load model
from joblib import load
aml_model = load(model_path + "/model.pkl")
aml_model

In [ ]:

# Look at model environment
with open(model_path +  '/./python_env.yaml') as f:
    print(f.read())

# Look at model requirements
with open(model_path +  '/./requirements.txt') as f:
    print(f.read())

In [ ]:
# connect to Snowflake with Snowpark
import pandas as pd
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
from snowflake.ml.registry import registry

connection_parameters = {
    "account": "<locator>", # e.g. xy12345.us-east-2.aws 
    "user": "<username>", 
    "password": "<password>",
    "role": "<role>",
    "warehouse": "<vistual warehouse>",
    "database": "<database>",
    "schema": "<schema>"
    }
session = Session.builder.configs(connection_parameters).create()

In [ ]:

# connect to Snowpark registry and log model
reg = registry.Registry(session=session)
reg.log_model(aml_model, model_name='<name of model in Snowflake>', version_name='v1', sample_input_data=<sample dataframe>)

# verify model deployment and view functions associated with the model
mv = reg.get_model('<name of model in Snowflake>').version('v1')
mv.show_functions()